# Piper TTS - Local Test

Bu notebook Piper TTS engine'ini lokal Windows ortamında test eder.

**Engine:** Piper (CPU-only)
**GPU:** Gerekmez
**Türkçe Desteği:** Evet (tr_TR-dfki-medium model)

Her soru için hem SORU hem CEVAP seslendiriliyor.

In [ ]:
# Cell 1: Install Dependencies
print("Installing dependencies...")

!pip install piper-tts
!pip install -e ..

print("\nVerifying installation...")
import tts_ms
print(f"tts_ms installed: {tts_ms.__file__}")

In [ ]:
# Cell 2: Setup Directories & Download Model
import os
from pathlib import Path
import urllib.request

PROJECT_DIR = Path("..").resolve()
OUTPUT_DIR = PROJECT_DIR / "output" / "piper_local"
AUDIO_DIR = OUTPUT_DIR / "audio"
CACHE_DIR = PROJECT_DIR / "cache" / "piper"

for d in [AUDIO_DIR, CACHE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Project: {PROJECT_DIR}")
print(f"Output: {OUTPUT_DIR}")

# Download Piper Turkish model
MODEL_NAME = "tr_TR-dfki-medium"
MODEL_PATH = CACHE_DIR / f"{MODEL_NAME}.onnx"
CONFIG_PATH = CACHE_DIR / f"{MODEL_NAME}.onnx.json"

BASE_URL = "https://huggingface.co/rhasspy/piper-voices/resolve/main/tr/tr_TR/dfki/medium"

if not MODEL_PATH.exists():
    print(f"\nDownloading {MODEL_NAME}...")
    urllib.request.urlretrieve(f"{BASE_URL}/{MODEL_NAME}.onnx", MODEL_PATH)
    urllib.request.urlretrieve(f"{BASE_URL}/{MODEL_NAME}.onnx.json", CONFIG_PATH)
    print("Model downloaded.")
else:
    print(f"\nModel cached: {MODEL_PATH}")

os.environ["TTS_MODEL_TYPE"] = "piper"
os.environ["TTS_MS_LOG_LEVEL"] = "4"
os.environ["TTS_MS_RESOURCES_ENABLED"] = "1"

In [ ]:
# Cell 3: Initialize TTSService
import time
import uuid
from tts_ms.services import TTSService
from tts_ms.services.tts_service import SynthesizeRequest
from tts_ms.core.config import Settings

print("Initializing TTSService...")
start = time.time()

settings = Settings(raw={
    'tts': {
        'engine': 'piper',
        'device': 'cpu',
        'piper': {
            'model_path': str(MODEL_PATH),
            'config_path': str(CONFIG_PATH)
        }
    },
    'logging': {'level': 4}
})
service = TTSService(settings)
init_time = time.time() - start
print(f"Initialized in {init_time:.2f}s")

# Warmup
print("\nWarmup...")
start = time.time()
_ = service.synthesize(SynthesizeRequest(text="Test."), request_id="warmup")
warmup_time = time.time() - start
print(f"Warmup done in {warmup_time:.2f}s")

In [ ]:
# Cell 4: Synthesize Questions & Answers
import time
import uuid
from IPython.display import Audio, display, HTML

QUESTIONS = [
    {"id": "01", "question": "Sizi neden işe almalıyız?",
     "answer": "Güçlü analitik düşünme becerilerim ve takım çalışmasına yatkınlığım sayesinde projelere değer katabilirim. Ayrıca sürekli öğrenmeye açık yapım ve problem çözme yeteneklerim, şirketinizin hedeflerine ulaşmasında önemli katkılar sağlayacaktır."},
    {"id": "02", "question": "Siz bizi neden seçtiniz?",
     "answer": "Şirketinizin yenilikçi yaklaşımı ve sektördeki lider konumu beni çok etkiledi. Kariyer hedeflerimle örtüşen bu ortamda kendimi geliştirebileceğime ve anlamlı projeler üzerinde çalışabileceğime inanıyorum."},
    {"id": "03", "question": "Kötü özellikleriniz nelerdir?",
     "answer": "Bazen aşırı detaycı olabiliyorum, bu da zaman yönetimimi olumsuz etkileyebiliyor. Ancak bu özelliğimin farkındayım ve önceliklendirme teknikleri kullanarak bu durumu yönetmeye çalışıyorum."},
    {"id": "04", "question": "Beş yıl sonra kendinizi nerede görüyorsunuz?",
     "answer": "Beş yıl içinde teknik liderlik pozisyonunda olmayı hedefliyorum. Ekip yönetimi deneyimi kazanarak şirketin büyümesine stratejik katkılar sağlamak istiyorum."},
    {"id": "05", "question": "Maaş beklentiniz nedir?",
     "answer": "Piyasa koşullarını ve pozisyonun gerekliliklerini değerlendirerek, deneyimime ve yeteneklerime uygun rekabetçi bir maaş beklentim var. Bu konuda esnek olmaya ve karşılıklı bir anlaşmaya varmaya açığım."}
]

results = []

print(f"{'='*60}")
print(f"SYNTHESIZING {len(QUESTIONS)} QUESTIONS + ANSWERS")
print(f"{'='*60}\n")

for q in QUESTIONS:
    print(f"\n[{q['id']}] {q['question']}")
    
    # === SORU ===
    print(f"  SORU: ", end="")
    start = time.time()
    try:
        result = service.synthesize(SynthesizeRequest(text=q['question']), request_id=f"{q['id']}_soru")
        elapsed = time.time() - start
        
        soru_path = AUDIO_DIR / f"{q['id']}_soru.wav"
        with open(soru_path, 'wb') as f:
            f.write(result.wav_bytes)
        
        results.append({'id': q['id'], 'type': 'soru', 'text': q['question'],
                       'time': elapsed, 'size': len(result.wav_bytes), 'status': 'OK'})
        print(f"{elapsed:.2f}s | {len(result.wav_bytes)/1024:.1f} KB | OK")
        display(Audio(str(soru_path)))
    except Exception as e:
        results.append({'id': q['id'], 'type': 'soru', 'text': q['question'],
                       'time': time.time()-start, 'size': 0, 'status': f'FAIL: {e}'})
        print(f"FAIL: {e}")
    
    # === CEVAP ===
    print(f"  CEVAP: ", end="")
    start = time.time()
    try:
        result = service.synthesize(SynthesizeRequest(text=q['answer']), request_id=f"{q['id']}_cevap")
        elapsed = time.time() - start
        
        cevap_path = AUDIO_DIR / f"{q['id']}_cevap.wav"
        with open(cevap_path, 'wb') as f:
            f.write(result.wav_bytes)
        
        results.append({'id': q['id'], 'type': 'cevap', 'text': q['answer'][:50],
                       'time': elapsed, 'size': len(result.wav_bytes), 'status': 'OK'})
        print(f"{elapsed:.2f}s | {len(result.wav_bytes)/1024:.1f} KB | OK")
        display(Audio(str(cevap_path)))
    except Exception as e:
        results.append({'id': q['id'], 'type': 'cevap', 'text': q['answer'][:50],
                       'time': time.time()-start, 'size': 0, 'status': f'FAIL: {e}'})
        print(f"FAIL: {e}")

successful = [r for r in results if r['status'] == 'OK']
print(f"\n{'='*60}")
print(f"COMPLETE: {len(successful)}/{len(results)} successful")
print(f"{'='*60}")

In [ ]:
# Cell 5: Summary
import json
from datetime import datetime

successful = [r for r in results if r['status'] == 'OK']
soru_results = [r for r in successful if r['type'] == 'soru']
cevap_results = [r for r in successful if r['type'] == 'cevap']

total_size_kb = sum(r['size'] for r in successful) / 1024
avg_soru = sum(r['time'] for r in soru_results) / len(soru_results) if soru_results else 0
avg_cevap = sum(r['time'] for r in cevap_results) / len(cevap_results) if cevap_results else 0

summary = f"""================================================
TTS-MS LOCAL TEST - PIPER
================================================
Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Platform: Windows (Local)
Model: {MODEL_NAME}

PERFORMANS:
- Init Time: {init_time:.2f}s
- Warmup Time: {warmup_time:.2f}s

SONUÇLAR:
- Toplam Soru: {len(QUESTIONS)}
- Soru Audio: {len(soru_results)}/5 başarılı (avg: {avg_soru:.2f}s)
- Cevap Audio: {len(cevap_results)}/5 başarılı (avg: {avg_cevap:.2f}s)
- Toplam Audio: {total_size_kb:.1f} KB

DETAYLAR:
"""
for r in results:
    status = 'OK' if r['status'] == 'OK' else 'FAIL'
    summary += f"[{r['id']}_{r['type']}] {r['time']:.2f}s | {r['size']/1024:.1f} KB | {status}\n"

summary += f"""
AUDIO DOSYALARI:
{AUDIO_DIR}
================================================
"""

with open(OUTPUT_DIR / 'summary.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

with open(OUTPUT_DIR / 'results.json', 'w', encoding='utf-8') as f:
    json.dump({'engine': 'piper', 'model': MODEL_NAME, 'results': results,
               'init_time': init_time, 'warmup_time': warmup_time}, f, indent=2, ensure_ascii=False)

print(summary)
print(f"\nFiles saved to: {OUTPUT_DIR}")